In [1]:
#import library yang dibituhkan
# ML menggunakan scipy 
# Scipy berfungsi untuk memodelkan fungsi-fungsi matematika
# dalam hal ini untuk melakukan optimisasi
import numpy as np
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import SeqIO
from scipy.optimize import differential_evolution
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

/Users/ed282828/Library/Python/3.9/lib/python/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
#membaca file fasta dan menyimpannya dalam variabel
apoeAcipenser = list(SeqIO.parse("APOE-Acipenser.fasta", "fasta"))
apoeBos = list(SeqIO.parse("APOE-Bos.fasta", "fasta"))
apoeCercopithecus = list(SeqIO.parse("APOE-Cercopithecus.fasta", "fasta"))
apoeOryctolagus = list(SeqIO.parse("APOE-Oryctolagus.fasta", "fasta"))
apoePan = list(SeqIO.parse("APOE-Pan.fasta", "fasta"))
apoeSiniperca = list(SeqIO.parse("APOE-Siniperca.fasta", "fasta"))

In [3]:
#mengambil sequence dari file fasta
#sequence yang diambil hanya 138 karakter pertama
# hal ini dilakukan karena MSA hanya mendukung jumlah alignment yang sama
seqapoeAcipenser = apoeAcipenser[0].seq[:138]
seqapoeBos = apoeBos[0].seq[:138]
seqapoECercopithecus = apoeCercopithecus[0].seq[:138]
seqapoeOryctolagus= apoeOryctolagus[0].seq[:138]
seqapoePan = apoePan[0].seq[:138]
seqapoESiniperca = apoeSiniperca[0].seq[:138]

In [4]:
#membuat multiple sequence alignment
sequences = [
    SeqRecord(seqapoeAcipenser,id = 'Acipenser'),
    SeqRecord(seqapoeBos,id = 'Bos Taurus'),
    SeqRecord(seqapoECercopithecus,id = 'Cercoptithecus aethiops'),
    SeqRecord(seqapoeOryctolagus,id = 'Oryctolagus cuniculus'),
    SeqRecord(seqapoePan,id = 'Pan troglodytes'),
    SeqRecord(seqapoESiniperca,id = 'Siniperca chuatsi')  
]

In [5]:
#menyimpan hasil multiple sequence alignment dalam variabel
align = MultipleSeqAlignment(sequences)

In [6]:
#fungsi untuk meelihat daerah yang termutasi
def mutations(alignment, acipenser_seq):
    results = {}
    acipenser_seq_str = str(acipenser_seq)

    for record in alignment:
        if record.id != 'Acipenser':
            conserved = []
            mutations = []
            for i, (acipenser_res, other_res) in enumerate(zip(acipenser_seq_str, str(record.seq))):
                if acipenser_res == other_res:
                    conserved.append((i+1, acipenser_res))
                else:
                    mutations.append((i+1, acipenser_res, other_res))
            results[record.id] = {
                'conserved': conserved,
                'mutations': mutations
            }
    return results

In [7]:
#fungsi untuk melihat daerah konservatif
def conserved(alignment, acipenser_seq):
    results = {}
    acipenser_seq_str = str(acipenser_seq)

    for record in alignment:
        if record.id != 'Acipenser':
            comparison = []
            for acipenser_res, other_res in zip(acipenser_seq_str, str(record.seq)):
                if acipenser_res == other_res:
                    comparison.append(acipenser_res)
                else:
                    comparison.append('x')
            results[record.id] = ''.join(comparison)
    return results

In [8]:
#menampilkan hasil mutasi
results_mutation = mutations(align, seqapoeAcipenser)

# Tampilkan hasil perspecies
for species, data in results_mutation.items():
    print(f"\nHasil untuk {species}:")
    
    print("Mutasi:")
    for pos, acipenser_res, other_res in data['mutations']:
        print(f"Posisi {pos}: Acipenser ({acipenser_res}) -> {species} ({other_res})")


Hasil untuk Bos Taurus:
Mutasi:
Posisi 2: Acipenser (G) -> Bos Taurus (C)
Posisi 3: Acipenser (A) -> Bos Taurus (G)
Posisi 6: Acipenser (A) -> Bos Taurus (G)
Posisi 8: Acipenser (T) -> Bos Taurus (C)
Posisi 9: Acipenser (T) -> Bos Taurus (G)
Posisi 10: Acipenser (G) -> Bos Taurus (A)
Posisi 11: Acipenser (G) -> Bos Taurus (A)
Posisi 12: Acipenser (A) -> Bos Taurus (G)
Posisi 13: Acipenser (A) -> Bos Taurus (G)
Posisi 14: Acipenser (C) -> Bos Taurus (A)
Posisi 16: Acipenser (C) -> Bos Taurus (G)
Posisi 17: Acipenser (T) -> Bos Taurus (C)
Posisi 18: Acipenser (A) -> Bos Taurus (G)
Posisi 19: Acipenser (C) -> Bos Taurus (G)
Posisi 20: Acipenser (A) -> Bos Taurus (T)
Posisi 21: Acipenser (C) -> Bos Taurus (T)
Posisi 22: Acipenser (C) -> Bos Taurus (T)
Posisi 23: Acipenser (G) -> Bos Taurus (C)
Posisi 24: Acipenser (G) -> Bos Taurus (T)
Posisi 25: Acipenser (A) -> Bos Taurus (G)
Posisi 26: Acipenser (A) -> Bos Taurus (G)
Posisi 28: Acipenser (T) -> Bos Taurus (G)
Posisi 29: Acipenser (A) -

In [9]:
# Lakukan perbandingan
results_conserve = conserved(align, seqapoeAcipenser)

# Tampilkan hasil
for species, comparison in results_conserve.items():
    print(f"\nHasil untuk {species}:")
    print(comparison)


Hasil untuk Bos Taurus:
CxxGAxGxxxxxxxCxxxxxxxxxxxAxxxxAxxxAxxCxxxxxxxCAxxxGCxxGxCAGxxxxxxxxxGxTCTxxxxGxxGxxxxxxxGGxxxCxCxxxxxGCAxxxxxxCAGxxxGxxAxxGxxGxxxAxxxxGxx

Hasil untuk Cercoptithecus aethiops:
xxxxAxxxxxGxxCxxxxCxxCGxxxxxxxCxGxGxGxxCACCCxxCxGxxxxTxxGxxxxxxGxAxxxxxxxxxxxxxGAxxxTGxxGGxxxTxxxxxxxxxCAxxTCxxxAxTxxxGxAxxxGxxxxxxxxxxxxx

Hasil untuk Oryctolagus cuniculus:
CxxxxxxxxGGxxCxxxxxAxxxxxxxxAxxAGxxxxxxxACCxAxxxxxxxxxGxxxAxxxxGxxGxxGxxxxxxxTGxxGxxxGCxGxGAxxCTxxxAGxxxxCxxxxxCxxxAxGGxxxAGGAGxTxxAGxxGCx

Hasil untuk Pan troglodytes:
CxxxxxxxxxGAAxxxxxxxCxxxxxxxxxxxxxGAxTxxxCxxxxxxxxxxxxGxxxAxxxxGxxxCxGxxxxxxxxGxxGxxxGxCxxxxxxCTxxxAGxxxxCxxxxxCAxxxxGGxxxAxGxGxTxxAGAxxxA

Hasil untuk Siniperca chuatsi:
xxxxAxxTxxxxxCxxTAxxCCxxxxxxxxxxxxxxxTCxxxxxxxxxxAxGCxxGxxxxxTxxCxxxxGATxxxxxxxxxxxxTxCxGGGAGxxxxCAxxxGxxxxxxxxxAGxAxxxxxxxxxAGxTCxxxAxxxx


In [10]:
# sequence yang akan dijadikan sebagai target
# sequence dijakian string untuk mempermudah proses
# alasan memotong sequence untuk memperceepat pemodelan :)
sequences = [
    str(apoeAcipenser[0].seq[:138]),
    str(apoeBos[0].seq[:138]),
    str(apoeCercopithecus[0].seq[:138]),
    str(apoeOryctolagus[0].seq[:138]),
    str(apoePan[0].seq[:138]),
    str(apoeSiniperca[0].seq[:138])
]

In [11]:
#fungsi untuk menghitung skor alignment
#berguna untuk kperhitungan optimisasi
def alignment_score(params):
    gap_open_penalty, gap_extend_penalty = params
    if gap_open_penalty < gap_extend_penalty:
        return np.inf  # invalid parameter combination
    total_score = 0
    num_alignments = 0
    for i in range(len(sequences)):
        for j in range(i + 1, len(sequences)):
            alignments = pairwise2.align.globalxs(sequences[i], sequences[j], -gap_open_penalty, -gap_extend_penalty)
            total_score += alignments[0].score  # Add the score of the best alignment
            num_alignments += 1
    average_score = total_score / num_alignments
    return -average_score  # Negative because differential_evolution minimizes the function

In [12]:
# menentukan nilai batas untuk gap open dan gap extend
bounds = [(0.1, 10.0), (0.1, 10.0)]

# melakukan pergitungan optimisasi
result = differential_evolution(alignment_score, bounds)

#mendapatkan hasil optimisasi untuk gap open dan gap extend
best_gap_open_penalty, best_gap_extend_penalty = result.x

In [13]:
#menampilkan hasil optimisasi untuk gap open dan gap extend`
print(f"Best gap open penalty: {best_gap_open_penalty}")
print(f"Best gap extend penalty: {best_gap_extend_penalty}")

Best gap open penalty: 0.1
Best gap extend penalty: 0.1


In [14]:
# mencetak hasil alignment terbaik
best_alignments = pairwise2.align.globalxs(sequences[0], sequences[1], -best_gap_open_penalty, -best_gap_extend_penalty)
print(format_alignment(*best_alignments[0]))

C-GAGAAGTTG-GAACCCTACACCGGAAATATCAGAGAGTCCACCCAGCAGAAGCTG--GGC-A---GCTTGC-AGCAGATC----TTCT-TGGAGAGTGCCGGGAGTC-TCCAAGAGCACTTCAA-CAGTAAGGCAGA-GGAGT-TCAAG--A-AGCA----
| | | ||  | |||  ..|| .|||   |.||.| |||.|.|   || | |.||.|  ||| |   ||..|| ||.||||.    |||| ||| | .|||||.| ||. | |    ||.||||.. |||.|.|.|||. ||| | |..||  | |||.    
CCG-G-AG--GCGAA--GGAC-GCGG---TTTCTG-GAGGCGA---AG-A-ACGCGGTTGGCCAATCGCAAGCCAGAAGATGAAGGTTCTGTGG-G-TTGCCGTG-GTGGT-C----GCGCTTCTGGCAGGATGCCAGGCGGA-TATGGAGGGAGAGCTGGGG
  Score=82



In [15]:
# mencetak hasil alignment terbaik
# ini adalah garis pertama dari alignment terbaik dari semua species terhadap Acipenser
first_alignment_line = format_alignment(*best_alignments[0]).split('\n')[0]
print("First line of the best alignment:")
print(first_alignment_line)

First line of the best alignment:
C-GAGAAGTTG-GAACCCTACACCGGAAATATCAGAGAGTCCACCCAGCAGAAGCTG--GGC-A---GCTTGC-AGCAGATC----TTCT-TGGAGAGTGCCGGGAGTC-TCCAAGAGCACTTCAA-CAGTAAGGCAGA-GGAGT-TCAAG--A-AGCA----
